In [141]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from finta import TA
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import jaccard_score
import seaborn as sn
from tabulate import tabulate
from xgboost import XGBClassifier
from ta import add_all_ta_features
from sklearn.feature_selection import RFE
import xgboost as xgb

In [142]:
WINDOW = 16  # number of rows to look ahead to see what the price did
FETCH_INTERVAL = "60m"  # fetch data by interval (including intraday if period < 60 days)
# valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
# (optional, default is '1d')
INTERVAL = '2y'  # use "period" instead of start/end
# valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# (optional, default is '1mo')
symbol = 'AAPL'  # Symbol of the desired stock

# one day 16 rows of data

In [143]:
data = yf.download(  # or pdr.get_data_yahoo(...
    tickers=symbol,

    period=INTERVAL,

    interval=FETCH_INTERVAL,

    # group by ticker (to access via data['SPY'])
    # (optional, default is 'column')
    group_by='ticker',

    # adjust all OHLC automatically
    # (optional, default is False)
    # auto_adjust = True,

    # download pre/post regular market hours data
    # (optional, default is False)
    prepost=True,

    # use threads for mass downloading? (True/False/Integer)
    # (optional, default is True)
    threads=True,

    # proxy URL scheme use use when downloading?
    # (optional, default is None)
    proxy=None
)
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2019-08-21 04:00:00-04:00,52.775000,53.110000,52.775000,53.050000,53.050000,0
2019-08-21 05:00:00-04:00,53.045000,53.150000,53.045000,53.150000,53.150000,0
2019-08-21 06:00:00-04:00,53.152500,53.200000,53.045000,53.080000,53.080000,0
2019-08-21 07:00:00-04:00,53.112500,53.112500,53.000000,53.070000,53.070000,0
2019-08-21 08:00:00-04:00,53.050000,53.100000,53.000000,53.065000,53.065000,0
...,...,...,...,...,...,...
2021-08-20 15:30:00-04:00,148.087006,148.289993,147.945007,148.190002,148.190002,6188690
2021-08-20 16:00:00-04:00,148.190000,148.300000,147.616000,148.190000,148.190000,0
2021-08-20 17:00:00-04:00,148.170000,149.580000,142.382000,148.200000,148.200000,0


In [144]:
# data = pd.read_csv('C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\preprocess_new\\AAPL_2y_8_11_05_2021 09_00_35_full.csv', index_col='Datetime')
# data

In [145]:
data.rename(columns={"Close": 'close', "High": 'high', "Low": 'low', 'Volume': 'volume', 'Open': 'open'}, inplace=True)
data.head(10)
important_columns = ['open', 'high', 'low', 'close', 'volume']

In [146]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8379 entries, 2019-08-21 04:00:00-04:00 to 2021-08-20 19:00:00-04:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       8379 non-null   float64
 1   high       8379 non-null   float64
 2   low        8379 non-null   float64
 3   close      8379 non-null   float64
 4   Adj Close  8379 non-null   float64
 5   volume     8379 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 458.2 KB


In [147]:

def calculate_diffs(diff_number, col_name):
    new_col_name = f'{col_name}_{diff_number}'
    data[new_col_name] = data[col_name].diff(diff_number)

In [148]:
# for name in important_columns:
#     for i in range(1, 11):
#         calculate_diffs(i, name)
#
# data.head(10)

In [149]:
data = data.dropna()

In [150]:
# data
# data.set_index(pd.DatetimeIndex(data['Datetime']))
# data

In [151]:
# fig, ax = plt.subplots()
#
# data.plot(kind='line',x='Datetime',y='close',ax=ax)
#
# every_nth = 4
# for n, label in enumerate(ax.xaxis.get_ticklabels()):
#     if n % every_nth != 0:
#         label.set_visible(False)
#
# plt.xticks(rotation=90)
# plt.show()

In [152]:
# import seaborn as sns
# sns.set()
# ax = data['open'].plot()
# plt.xticks(rotation=90)
# plt.ylabel('Kurs akcji [$]', rotation=90)
# plt.xlabel('Czas')
# plt.legend(['Close'], ncol=1, loc='upper left');
# plt.show()
#
# fig = ax.get_figure()
# fig.savefig('asdf.png', dpi=300, bbox_inches="tight")
#

In [153]:
# data['close_pct'] = data['close'].pct_change()
# data.head()

In [154]:
data.describe()

,open,high,low,close,Adj Close,volume
count,8379.000000,8379.000000,8379.000000,8379.000000,8379.000000,8.379000e+03
mean,101.062840,101.480012,100.557169,101.067265,101.067265,3.807779e+06
std,29.988389,30.351178,29.818943,29.985327,29.985327,6.934091e+06
min,50.250000,50.467500,50.250000,50.357500,50.357500,0.000000e+00
25%,71.579998,71.959999,71.220000,71.628689,71.628689,0.000000e+00
50%,111.450000,111.987503,110.599998,111.420000,111.420000,0.000000e+00
75%,126.925002,127.257450,126.381002,126.922502,126.922502,5.485080e+06
max,151.410000,438.440000,151.290000,151.670000,151.670000,9.845401e+07


In [155]:
data

,open,high,low,close,Adj Close,volume
Datetime,,,,,,
2019-08-21 04:00:00-04:00,52.775000,53.110000,52.775000,53.050000,53.050000,0
2019-08-21 05:00:00-04:00,53.045000,53.150000,53.045000,53.150000,53.150000,0
2019-08-21 06:00:00-04:00,53.152500,53.200000,53.045000,53.080000,53.080000,0
2019-08-21 07:00:00-04:00,53.112500,53.112500,53.000000,53.070000,53.070000,0
2019-08-21 08:00:00-04:00,53.050000,53.100000,53.000000,53.065000,53.065000,0
...,...,...,...,...,...,...
2021-08-20 15:30:00-04:00,148.087006,148.289993,147.945007,148.190002,148.190002,6188690
2021-08-20 16:00:00-04:00,148.190000,148.300000,147.616000,148.190000,148.190000,0
2021-08-20 17:00:00-04:00,148.170000,149.580000,142.382000,148.200000,148.200000,0


In [156]:
data.columns


Index(['open', 'high', 'low', 'close', 'Adj Close', 'volume'], dtype='object')

In [157]:
def create_class_column(row, lowest_threshold, higher_threshold):
    if row['close_shift'] - row['close'] > higher_threshold:
        return 1
    if row['close_shift'] - row['close'] < lowest_threshold:
        return -1
    else:
        return 0

In [158]:
important_columns = ['open', 'high', 'low', 'close']


def calculate_diffs(diff_number, col_name):
    new_col_name = f'{col_name}_{diff_number}'
    data[new_col_name] = data[col_name].diff(diff_number)


for name in important_columns:
    for i in range(1, 30):
        calculate_diffs(i, name)

data['close_shift'] = data.shift(-WINDOW)['close']
data

,open,high,low,close,Adj Close,volume,open_1,open_2,open_3,open_4,...,close_21,close_22,close_23,close_24,close_25,close_26,close_27,close_28,close_29,close_shift
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-08-21 04:00:00-04:00,52.775000,53.110000,52.775000,53.050000,53.050000,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.3475
2019-08-21 05:00:00-04:00,53.045000,53.150000,53.045000,53.150000,53.150000,0,0.270000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.2000
2019-08-21 06:00:00-04:00,53.152500,53.200000,53.045000,53.080000,53.080000,0,0.107500,0.377500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.9025
2019-08-21 07:00:00-04:00,53.112500,53.112500,53.000000,53.070000,53.070000,0,-0.040000,0.067500,0.337500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0250
2019-08-21 08:00:00-04:00,53.050000,53.100000,53.000000,53.065000,53.065000,0,-0.062500,-0.102500,0.005000,0.275000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.1950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-20 15:30:00-04:00,148.087006,148.289993,147.945007,148.190002,148.190002,6188690,0.372009,0.175003,0.277512,0.337006,...,0.744995,1.760101,2.139999,2.910002,2.970002,3.530002,2.880002,3.440002,3.670002,NaN
2021-08-20 16:00:00-04:00,148.190000,148.300000,147.616000,148.190000,148.190000,0,0.102994,0.475004,0.277997,0.380506,...,1.229093,0.744993,1.760099,2.139997,2.910000,2.970000,3.530000,2.880000,3.440000,NaN
2021-08-20 17:00:00-04:00,148.170000,149.580000,142.382000,148.200000,148.200000,0,-0.020000,0.082994,0.455004,0.257997,...,2.110004,1.239093,0.754993,1.770099,2.149997,2.920000,2.980000,3.540000,2.890000,NaN


In [159]:
higher_threshold = 1.5
lowest_threshold = -1.5
last_values_higher = []
last_values_lower = []
data['class_column'] = data.apply((lambda x: create_class_column(x, lowest_threshold, higher_threshold)), axis=1)
while True:
    # print(data['class_column'].value_counts())
    class_counts = data['class_column'].value_counts()
    if abs(class_counts[0] - class_counts[1]) < 15 and abs(class_counts[0] - class_counts[-1]) < 15:
        break

    if len(last_values_higher) == 3:
        last_values_higher.pop(0)
    if len(last_values_lower) == 3:
        last_values_lower.pop(0)

    last_values_higher.append(higher_threshold)
    last_values_lower.append(lowest_threshold)
    if class_counts[0] > class_counts[1]:
        higher_threshold -= 0.01
    if class_counts[0] > class_counts[-1]:
        lowest_threshold += 0.01
    if class_counts[0] < class_counts[1]:
        higher_threshold += 0.01
    if class_counts[0] < class_counts[-1]:
        lowest_threshold -= 0.01

    if higher_threshold in last_values_higher and lowest_threshold in last_values_lower:
        break
    # print(higher_threshold, lowest_threshold)
    data['class_column'] = data.apply((lambda x: create_class_column(x, lowest_threshold, higher_threshold)), axis=1)

data

,open,high,low,close,Adj Close,volume,open_1,open_2,open_3,open_4,...,close_22,close_23,close_24,close_25,close_26,close_27,close_28,close_29,close_shift,class_column
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-08-21 04:00:00-04:00,52.775000,53.110000,52.775000,53.050000,53.050000,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.3475,0
2019-08-21 05:00:00-04:00,53.045000,53.150000,53.045000,53.150000,53.150000,0,0.270000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.2000,0
2019-08-21 06:00:00-04:00,53.152500,53.200000,53.045000,53.080000,53.080000,0,0.107500,0.377500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.9025,0
2019-08-21 07:00:00-04:00,53.112500,53.112500,53.000000,53.070000,53.070000,0,-0.040000,0.067500,0.337500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0250,0
2019-08-21 08:00:00-04:00,53.050000,53.100000,53.000000,53.065000,53.065000,0,-0.062500,-0.102500,0.005000,0.275000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.1950,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-20 15:30:00-04:00,148.087006,148.289993,147.945007,148.190002,148.190002,6188690,0.372009,0.175003,0.277512,0.337006,...,1.760101,2.139999,2.910002,2.970002,3.530002,2.880002,3.440002,3.670002,NaN,0
2021-08-20 16:00:00-04:00,148.190000,148.300000,147.616000,148.190000,148.190000,0,0.102994,0.475004,0.277997,0.380506,...,0.744993,1.760099,2.139997,2.910000,2.970000,3.530000,2.880000,3.440000,NaN,0
2021-08-20 17:00:00-04:00,148.170000,149.580000,142.382000,148.200000,148.200000,0,-0.020000,0.082994,0.455004,0.257997,...,1.239093,0.754993,1.770099,2.149997,2.920000,2.980000,3.540000,2.890000,NaN,0


In [160]:
# Class divide
data['class_column'] = data.apply((lambda x: create_class_column(x, lowest_threshold, higher_threshold)), axis=1)

data['class_column'].value_counts()

 0    2817
 1    2804
-1    2758
Name: class_column, dtype: int64

In [161]:
filename_to_export = f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test\\{symbol}_{INTERVAL}_{WINDOW}_diff_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}_full.csv'
data.to_csv(filename_to_export, index=True)
print(filename_to_export)

C:\Users\exomat\Desktop\repo\magisterka_analiza\data\results\train_test\AAPL_2y_16_diff_21_08_2021 12_52_42_full.csv


In [162]:

# Class divide
data['class_column'].value_counts()

 0    2817
 1    2804
-1    2758
Name: class_column, dtype: int64

In [163]:
# del (data['close'])
del (data['close_shift'])
data = data.dropna()

In [164]:
data['class_column'].value_counts()


 1    2804
 0    2794
-1    2752
Name: class_column, dtype: int64

In [165]:
y = data['class_column']
features = [x for x in data.columns if x not in ['class_column']]
x = data[features]
# scaler = MinMaxScaler()
# x = pd.DataFrame(scaler.fit_transform(x.values), columns=x.columns, index=x.index)
ROWS_TO_PREDICT = 64
if ROWS_TO_PREDICT == 33:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=False)
else:
    x_train = x.iloc[:-ROWS_TO_PREDICT]
    y_train = y.iloc[:-ROWS_TO_PREDICT]
    x_test = x.iloc[-ROWS_TO_PREDICT:]
    y_test = y.iloc[-ROWS_TO_PREDICT:]

In [166]:
classifiers = dict()
classifiers = dict()
classifiers['DecisionTreeClassifier 1'] = DecisionTreeClassifier(max_depth=15, random_state=0, criterion='gini',
                                                                 splitter='best')
classifiers['DecisionTreeClassifier 2'] = DecisionTreeClassifier(max_depth=40, random_state=0, criterion='gini',
                                                                 splitter='best')
classifiers['DecisionTreeClassifier 3'] = DecisionTreeClassifier(max_depth=10, random_state=0, criterion='gini',
                                                                 splitter='best')

classifiers['RandomForestClassifier 1'] = RandomForestClassifier(n_estimators=1000, max_depth=3, random_state=0,
                                                                 criterion='gini', n_jobs=-1)

classifiers['RandomForestClassifier 2'] = RandomForestClassifier(n_estimators=1000, max_depth=3, random_state=0,
                                                                 criterion='entropy', n_jobs=-1)

classifiers['RandomForestClassifier 3'] = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0,
                                                                 criterion='entropy', n_jobs=-1)

classifiers['GradientBoostingClassifier 1'] = GradientBoostingClassifier(n_estimators=100, random_state=0,
                                                                         criterion='friedman_mse', max_depth=3,
                                                                         learning_rate=0.1)

classifiers['GradientBoostingClassifier 2'] = GradientBoostingClassifier(n_estimators=1000, random_state=0,
                                                                         criterion='friedman_mse', max_depth=3,
                                                                         learning_rate=0.5)

classifiers['XGBRFClassifier 1'] = xgb.sklearn.XGBRFClassifier(n_jobs=-1, max_depth=2, n_estimators=1000, eta=0.2)
classifiers['XGBRFClassifier 2'] = xgb.sklearn.XGBRFClassifier(n_jobs=-1, max_depth=6, n_estimators=1000, eta=0.3)

classifiers['XGBClassifier 1'] = xgb.XGBClassifier(nthread=-1, max_depth=3, n_estimators=1000, eta=0.2)
classifiers['XGBClassifier 2'] = xgb.XGBClassifier(nthread=-1, max_depth=3, n_estimators=1000, eta=0.3)

classifiers_boosted = dict()
classifiers_boosted['SGradientBoostingClassifier 1'] = GradientBoostingClassifier(n_estimators=100, random_state=0,
                                                                                  criterion='friedman_mse', max_depth=3,
                                                                                  learning_rate=0.1)
classifiers_boosted['SXGBClassifier 1'] = xgb.XGBClassifier(nthread=-1, max_depth=3, n_estimators=1000, eta=0.2)
classifiers_boosted['SXGBClassifier 2'] = xgb.XGBClassifier(nthread=-1, max_depth=3, n_estimators=1000, eta=0.3)
classifiers_boosted['SXGBClassifier 3'] = xgb.XGBClassifier(nthread=-1, max_depth=10, n_estimators=1000, eta=0.2)

In [167]:
def print_conf_matrix(test_y, predict, name):
    matrix = confusion_matrix(test_y, predict, labels=[-1, 0, 1])
    print(matrix)
    ax = plt.subplot()
    sn.heatmap(matrix, annot=True, ax=ax)  #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted labels', color='white')
    ax.set_ylabel('True labels', color='white')
    ax.set_title(f'Confusion Matrix for {name}', color='white')
    ax.xaxis.set_ticklabels(['-1', '0', '1'], color='white')
    ax.yaxis.set_ticklabels(['-1', '0', '1'], color='white')
    plt.show()


def train_model(model, train_x, train_y):
    model.fit(train_x, train_y)


In [168]:
predictions_train = dict()
score_train = dict()
predictions = dict()
score = dict()

In [169]:

for k, v in classifiers.items():
    print("Calculate: ", k)
    train_model(v, x_train, y_train)
    predictions_train[k] = v.predict(x_train)
    score_train[k] = accuracy_score(y_train.values, predictions_train[k])
    predictions[k] = v.predict(x_test)
    score[k] = accuracy_score(y_test.values, predictions[k])
    print('Score_train: ', score_train[k])
    print('Score: ', score[k])
    # print_conf_matrix(test_y, predictions[k], k)


Calculate:  DecisionTreeClassifier 1
Score_train:  0.8580738595220855
Score:  0.515625
Calculate:  DecisionTreeClassifier 2
Score_train:  1.0
Score:  0.484375
Calculate:  DecisionTreeClassifier 3
Score_train:  0.647598358677287
Score:  0.375
Calculate:  RandomForestClassifier 1
Score_train:  0.48237991793386437
Score:  0.421875
Calculate:  RandomForestClassifier 2
Score_train:  0.4798455225681873
Score:  0.4375
Calculate:  RandomForestClassifier 3
Score_train:  0.9691045136374608
Score:  0.40625
Calculate:  GradientBoostingClassifier 1
Score_train:  0.627564566739078
Score:  0.515625
Calculate:  GradientBoostingClassifier 2
Score_train:  1.0
Score:  0.453125
Calculate:  XGBRFClassifier 1


C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:05:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Score_train:  0.45836350470673426
Score:  0.4375
Calculate:  XGBRFClassifier 2
[13:06:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Score_train:  0.6101858556601496
Score:  0.40625
Calculate:  XGBClassifier 1
[13:06:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merr

In [170]:
# filename_to_export = f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test\\result_test_xrand_{symbol}_{WINDOW}_{ROWS_TO_PREDICT}_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}.csv'
# filename_to_export_train= f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test\\result_train_xrand_{symbol}_{WINDOW}_{ROWS_TO_PREDICT}_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}.csv'
#
# headers = ["Classifier type", "Accuracy"]
# score_df = pd.DataFrame(score.items(), columns=headers)
# print(tabulate(score_df, headers, tablefmt="psql"))
# score_df.to_csv(filename_to_export,mode='a', index=False, header=False)
# score_df_train = pd.DataFrame(score_train.items(), columns=headers)
# print(tabulate(score_df_train, headers, tablefmt="psql"))
# score_df_train.to_csv(filename_to_export_train,mode='a', index=False, header=False)

In [171]:
rfe = RFE(classifiers['XGBRFClassifier 1'], 10)
fited = rfe.fit(x_train, y_train)
rfe

C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:07:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:07:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:25:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:25:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RFE(estimator=XGBRFClassifier(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bytree=1, eta=0.2,
                              gamma=0, gpu_id=-1, importance_type='gain',
                              interaction_constraints='', max_delta_step=0,
                              max_depth=2, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=1000,
                              n_jobs=-1, num_parallel_tree=1000,
                              objective='multi:softprob', random_state=0,
                              reg_alpha=0, scale_pos_weight=None,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None),
    n_features_to_select=10)

In [172]:
names = x.columns
columns = []
for i in range(len(fited.support_)):
    if fited.support_[i]:
        columns.append(names[i])

print("Columns with predictive power:", columns)
columns = columns + ['high', 'low', 'volume', 'open']
columns

Columns with predictive power: ['high', 'close', 'Adj Close', 'open_12', 'high_4', 'high_12', 'high_13', 'close_2', 'close_4', 'close_11']


['high',
 'close',
 'Adj Close',
 'open_12',
 'high_4',
 'high_12',
 'high_13',
 'close_2',
 'close_4',
 'close_11',
 'high',
 'low',
 'volume',
 'open']

In [173]:
x_test_cropped = x_test[columns]
x_train_cropped = x_train[columns]
x_train_cropped
x_test_cropped = x_test_cropped.loc[:, ~x_test_cropped.columns.duplicated()]
x_train_cropped = x_train_cropped.loc[:, ~x_train_cropped.columns.duplicated()]
#

In [174]:
for k, v in classifiers_boosted.items():
    print("Calculate: ", k)
    train_model(v, x_train_cropped, y_train)
    predictions_train[k] = v.predict(x_train_cropped)
    score_train[k] = accuracy_score(y_train.values, predictions_train[k])
    predictions[k] = v.predict(x_test_cropped)
    score[k] = accuracy_score(y_test.values, predictions[k])
    print('Score train: ', score_train[k])
    print('Score: ', score[k])
    # print_conf_matrix(test_y, predictions[k], k)

filename_to_export = f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test_diff\\result_test_diff_{symbol}_{WINDOW}_{ROWS_TO_PREDICT}_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}.csv'
filename_to_export_train = f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test_diff\\result_train_diff_{symbol}_{WINDOW}_{ROWS_TO_PREDICT}_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}.csv'

headers = ["Classifier type", "Accuracy"]
score_df = pd.DataFrame(score.items(), columns=headers)
print(tabulate(score_df, headers, tablefmt="psql"))
score_df.to_csv(filename_to_export, mode='a', index=False, header=False)
score_df_train = pd.DataFrame(score_train.items(), columns=headers)
print(tabulate(score_df_train, headers, tablefmt="psql"))
score_df_train.to_csv(filename_to_export_train, mode='a', index=False, header=False)

Calculate:  SGradientBoostingClassifier 1
Score train:  0.5832729905865315
Score:  0.515625
Calculate:  SXGBClassifier 1
[13:25:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Score train:  0.9314506396331161
Score:  0.53125
Calculate:  SXGBClassifier 2
[13:25:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Score train:  0.9751387883176442
Score:  0.546875
Calculate:  SXGBClassifier 3
[13:25:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Score train:  1.0
Score:  0.53125
+----+-------------------------------+------------+
|    | Classifier type               |   Accuracy |
|----+-------------------------------+------------|
|  0 | DecisionTreeClassifier 1

In [175]:
x_train_cropped.columns

Index(['high', 'close', 'Adj Close', 'open_12', 'high_4', 'high_12', 'high_13',
       'close_2', 'close_4', 'close_11', 'low', 'volume', 'open'],
      dtype='object')

In [176]:
#
# model = xgb.XGBClassifier(nthread =-1,max_depth=14,n_estimators=1000,
#                           eta =0.2)
# model.fit(x_train,y_train)
# predicted_train = model.predict(x_train)
# predicted_test = model.predict(x_test)
# print("------------")
# print(f'max_depth: {14}')
# score_train['XGBClassifier'] = accuracy_score(y_train.values, predicted_train)
# score['XGBClassifier'] = accuracy_score(y_test.values, predicted_test)
# print(score_train['XGBClassifier'])
# print(score['XGBClassifier'])
# print("------------")
#

In [177]:
# model = xgb.sklearn.XGBRFClassifier(n_jobs=-1,max_depth=12,n_estimators =100,eta=0.4)
# model.fit(x_train,y_train)
# predicted_train = model.predict(x_train)
# predicted_test = model.predict(x_test)
# print("------------")
# print(f'eta: ')
# score_train['XGBRFClassifier'] = accuracy_score(y_train.values, predicted_train)
# score['XGBRFClassifier'] = accuracy_score(y_test.values, predicted_test)
# print(accuracy_score(y_train.values, predicted_train))
# print(accuracy_score(y_test.values, predicted_test))
# print("------------")